In [2]:
from transformers import pipeline
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import time
from datasets import Dataset

/home/chome/documents/cours/s5/pfe/pfe_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-27 12:32:42.779005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740655962.860136    1401 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740655962.883301    1401 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 12:32:43.089776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perfor

In [3]:
# Initialisation du pipeline zero-shot classification

# Bart large mnli => plusieurs minutes d'inférence => pas terrible du tout 
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", )
# Use a pipeline as a high-level helper

# distilbart-mnli-12-1 => environ 3 secondes d'inférence pour un même texte, beaucoup plus rapide pour des résultats qui ne sont pas tant que ça moins bons

# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1")
# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")
# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-6")
# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-9")


Device set to use cuda:0


In [39]:
# Définition des 17 ODD comme labels candidats
# candidate_labels = [
#     "ODD 1: Pas de pauvreté",
#     "ODD 2: Faim Zéro",
#     "ODD 3: Bonne santé et bien-être",
#     "ODD 4: Éducation de qualité",
#     "ODD 5: Égalité entre les sexes",
#     "ODD 6: Eau propre et assainissement",
#     "ODD 7: Énergie propre et d'un coût abordable",
#     "ODD 8: Travail décent et croissance économique",
#     "ODD 9: Industrie, innovation et infrastructure",
#     "ODD 10: Inégalités réduites",
#     "ODD 11: Villes et communautés durables",
#     "ODD 12: Consommation et production responsables",
#     "ODD 13: Lutte contre le changement climatique",
#     "ODD 14: Vie aquatique",
#     "ODD 15: Vie terrestre",
#     "ODD 16: Paix, justice et institutions efficaces",
#     "ODD 17: Partenariats pour la réalisation des objectifs"
# ]

# candidate_labels = [
#     "Pas de pauvreté",
#     "Faim Zéro",
#     "Bonne santé et bien-être",
#     "Éducation de qualité",
#     "Égalité entre les sexes",
#     "Eau propre et assainissement",
#     "Énergie propre et d'un coût abordable",
#     "Travail décent et croissance économique",
#     "Industrie, innovation et infrastructure",
#     "Inégalités réduites",
#     "Villes et communautés durables",
#     "Consommation et production responsables",
#     "Lutte contre le changement climatique",
#     "Vie aquatique",
#     "Vie terrestre",
#     "Paix, justice et institutions efficaces",
#     "Partenariats pour la réalisation des objectifs"
# ]


# candidate_labels = [
#     "Pas de pauvreté : Éliminer la pauvreté sous toutes ses formes et partout dans le monde",
#     "Faim Zéro : Éliminer la faim, assurer la sécurité alimentaire, améliorer la nutrition et promouvoir une agriculture durable",
#     "Bonne santé et bien-être : Donner aux individus les moyens de vivre une vie saine et promouvoir le bien-être à tous les âges",
#     "Éducation de qualité : Veiller à ce que tous puissent suivre une éducation de qualité dans des conditions d'équité et promouvoir les possibilités d'apprentissage tout au long de la vie",
#     "Égalité entre les sexes : Réaliser l'égalité des sexes et autonomiser toutes les femmes et les filles",
#     "Eau propre et assainissement : Garantir l'accès de tous à l'eau et à l'assainissement et assurer une gestion durable des ressources en eau",
#     "Énergie propre et d'un coût abordable : Garantir l'accès de tous à des services énergétiques fiables, durables et modernes, à un coût abordable",
#     "Travail décent et croissance économique : Promouvoir une croissance économique soutenue, partagée et durable, le plein emploi productif et un travail décent pour tous",
#     "Industrie, innovation et infrastructure : Mettre en place une infrastructure résiliente, promouvoir une industrialisation durable qui profite à tous et encourager l'innovation",
#     "Inégalités réduites : Réduire les inégalités entre les pays et en leur sein",
#     "Villes et communautés durables : Faire en sorte que les villes et les établissements humains soient ouverts à tous, sûrs, résilients et durables",
#     "Consommation et production responsables : Établir des modes de consommation et de production durables",
#     "Lutte contre le changement climatique : Prendre d'urgence des mesures pour lutter contre les changements climatiques et leurs répercussions",
#     "Vie aquatique : Conserver et exploiter de manière durable les océans, les mers et les ressources marines aux fins du développement durable",
#     "Vie terrestre : Préserver et restaurer les écosystèmes terrestres",
#     "Paix, justice et institutions efficaces : Promouvoir l'avènement de sociétés pacifiques et ouvertes aux fins du développement durable",
#     "Partenariats pour la réalisation des objectifs : Partenariats pour la réalisation des objectifs"
# ]


candidate_labels = [
    "Pauvreté",
    "Faim",
    "Bonne santé",
    "Éducation",
    "Égalité entre les sexes",
    "Eau propre",
    "Énergie coût abordable",
    "Travail et croissance économique",
    "Industrie, innovation et infrastructure",
    "Inégalités réduites",
    "Villes durables",
    "Consommation et production responsables",
    "changement climatique",
    "Vie aquatique",
    "Vie terrestre",
    "Paix, justice et institutions efficaces",
    "Partenariats pour la réalisation des objectifs"
]

In [40]:
def classify_thesis(text):
    # Découper le texte en phrases
    sentences = sent_tokenize(text, language="french")
    
    # Stocker les résultats pour chaque phrase
    results = []
    nb_sentences = len(sentences)
    for sentence in sentences:
        res = classifier(sentence, candidate_labels, multi_label=True)
        results.append(res)
    return sentences, results, nb_sentences

In [ ]:
text = """
Test qui parle de vie marine et de pollution. Il y a des poissons dans l'eau. Il y a de la pollution dans l'eau. La vie aquatique est menacée par la pollution. Heureusement les états se sont engagés à protéger la vie aquatique.
"""


In [42]:
# res = classifier(text, candidate_labels, multi_label=True)
# print(res)

In [43]:

start_time = time.time()

sentences, results, nb_sentences = classify_thesis(text)

end_time = time.time()
inference_time = end_time - start_time

In [44]:
print(results)

[{'sequence': "\nLa logistique a connu un essor considérable dans les entreprises à travers de profondes transformations de pratiques de gestion, et d'évolutions de contexte économique et technologique.", 'labels': ['Consommation et production responsables', 'Partenariats pour la réalisation des objectifs', 'Pauvreté', 'Travail et croissance économique', 'Industrie, innovation et infrastructure', 'Faim', 'Villes durables', 'Vie terrestre', 'Énergie coût abordable', 'Eau propre', 'Vie aquatique', 'Inégalités réduites', 'Paix, justice et institutions efficaces', 'Égalité entre les sexes', 'Bonne santé', 'changement climatique', 'Éducation'], 'scores': [0.3242719769477844, 0.25047048926353455, 0.1780506670475006, 0.16419124603271484, 0.16395454108715057, 0.14699722826480865, 0.1294001191854477, 0.0783952996134758, 0.045937005430459976, 0.021393558010458946, 0.0133826257660985, 0.01281090546399355, 0.004973263014107943, 0.003532069968059659, 0.0017934668576344848, 0.0015094922855496407, 0.

In [45]:
# Seuil pour prendre en compte le score d'une phrase
threshold = 0.7

# Agrégation des scores par ODD et identification de la phrase la plus contributive
agg_scores = {label: 0.0 for label in candidate_labels}
best_sentence_for_label = {label: ("", 0.0) for label in candidate_labels}

for res in results:
    for label, score in zip(res["labels"], res["scores"]):
        if score >= threshold:
            agg_scores[label] += score
            # agg_scores[label] /= nb_sentences
            if score > best_sentence_for_label[label][1]:
                best_sentence_for_label[label] = (res["sequence"], score)

# Classer les ODD dans l'ordre d'importance
sorted_odd = sorted(agg_scores.items(), key=lambda item: item[1], reverse=True)

print("Scores agrégés par ODD (classés par importance) :")
for label, score in sorted_odd:
    print(f"{label} : {score:.3f}")

print("\nPhrase la plus contributive par ODD (ordre d'importance) :")
for label, _ in sorted_odd:
    sentence, score = best_sentence_for_label[label]
    print(f"{label} (score {score:.3f}) -> {sentence}")

print(f"\nTemps d'inférence : {inference_time:.2f} secondes")

Scores agrégés par ODD (classés par importance) :
Industrie, innovation et infrastructure : 4.278
Partenariats pour la réalisation des objectifs : 2.548
Vie terrestre : 2.416
changement climatique : 1.599
Travail et croissance économique : 0.818
Faim : 0.711
Consommation et production responsables : 0.706
Pauvreté : 0.000
Bonne santé : 0.000
Éducation : 0.000
Égalité entre les sexes : 0.000
Eau propre : 0.000
Énergie coût abordable : 0.000
Inégalités réduites : 0.000
Villes durables : 0.000
Vie aquatique : 0.000
Paix, justice et institutions efficaces : 0.000

Phrase la plus contributive par ODD (ordre d'importance) :
Industrie, innovation et infrastructure (score 0.937) -> Une nouvelle technologie disruptive dans l'industrie fait son apparition en 2008, la blockchain, permettant aux entreprises d'adresser les clés nécessaires aux nouveaux enjeux présents dans la supplie Chain.
Partenariats pour la réalisation des objectifs (score 0.917) -> Ceux-ci permettent de réels progrès à la fois